Visualization 1 (Enhanced): Event Impact Overview for All Windows

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- Visualization 1 (Enhanced): Event Impact Overview for All Windows ---

# Note: This script reads the result file from the previous analysis,
# not the original data file.
# Therefore, there's no need to adjust the paths for Gold, Nasdaq100, or SPY.
try:
    df_mean = pd.read_csv("./outcome/event_study_mean_results.csv")
except FileNotFoundError:
    print("Error: 'event_study_mean_results.csv' not found in the 'outcome' folder. "
          "Please run the main event study script first.")
    # It's recommended to comment out exit() in a Jupyter environment.
    # If running as a .py file, you can keep exit().
    # exit()

# --- Data Preparation ---
# Select a model, but this time we will use data from ALL windows.
# Check if df_mean was loaded successfully and is not empty.
if 'df_mean' in locals() and not df_mean.empty:
    model_to_plot = 'MM_SPY'
    df_plot_data = df_mean[df_mean['Model'] == model_to_plot]

    # Define a logical order for the x-axis and the hue (colors)
    event_group_order = sorted(df_plot_data['EventGroup'].unique())
    window_order = ['(-1,+1)', '(-5,+5)', '(-10,+10)']

    # --- Plotting ---
    # Use hue to represent the different event windows within each group.
    g = sns.catplot(
        data=df_plot_data,
        x='EventGroup',
        y='MeanCAR',
        hue='Window',          # Use color to distinguish windows
        hue_order=window_order,
        kind='bar',
        col='Asset',           # Facet by Asset
        col_wrap=2,
        order=event_group_order,
        palette='magma',       # Use a sequential color palette
        height=5,
        aspect=1.5,
        legend_out=True
    )

    # --- Chart Enhancement ---
    g.fig.suptitle(f'Mean CAR by Event Group Across Windows\nModel: {model_to_plot}', y=1.03)
    g.set_axis_labels("Event Group", "Mean CAR")
    g.set_titles("Asset: {col_name}")
    g.despine(left=True)

    # Add a horizontal line at y=0
    for ax in g.axes.flat:
        ax.axhline(0, ls='--', color='black', linewidth=0.8)
        ax.tick_params(axis='x', rotation=45)

    # Adjust legend
    g.add_legend(title='Event Window')

    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

Visualization 2 (Enhanced & Ordered): Significance Heatmaps for All Windows

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# --- Visualization 2 (Enhanced & Ordered): Significance Heatmaps for All Windows ---

# Note: This script reads the result file from the previous analysis,
# not the original data file.
# Therefore, there's no need to adjust the paths for Gold, Nasdaq100, or SPY.
try:
    df_mean = pd.read_csv("./outcome/event_study_mean_results.csv")
except FileNotFoundError:
    print("Error: 'event_study_mean_results.csv' not found in the 'outcome' folder. "
          "Please run the main event study script first.")
    # It's recommended to comment out exit() in a Jupyter environment.
    # exit()

# --- Data Preparation & Plotting Loop ---
# Check if df_mean was loaded successfully and is not empty
if 'df_mean' in locals() and not df_mean.empty:
    # Select a model to analyze
    model_to_plot = 'MM_SPY'
    df_model_data = df_mean[df_mean['Model'] == model_to_plot]

    # --- MODIFICATION: Define a fixed order for window display ---
    window_order = ['(-1,+1)', '(-5,+5)', '(-10,+10)']
    # --- END MODIFICATION ---

    # Function to convert p-values to significance stars
    def p_to_stars(p):
        if p <= 0.01: return '***'
        if p <= 0.05: return '**'
        if p <= 0.1: return '*'
        return ''

    # Loop through each event window in the specified order
    for window in window_order:
        # Filter data for the current window
        df_plot_data = df_model_data[df_model_data['Window'] == window]
        
        if df_plot_data.empty:
            print(f"Warning: No data found for window {window}. Skipping this heatmap.")
            continue

        # Pivot the data for the heatmap
        try:
            heatmap_data = df_plot_data.pivot_table(index='Asset', columns='EventGroup', values='MeanCAR')
            p_value_data = df_plot_data.pivot_table(index='Asset', columns='EventGroup', values='p-value') # Corrected column name to 'p-value'
            annotations = p_value_data.applymap(p_to_stars)
        except Exception as e:
            print(f"Could not create pivot table for window {window}. Error: {e}")
            continue

        # --- Plotting ---
        plt.figure(figsize=(16, 8))
        sns.heatmap(
            heatmap_data,
            annot=annotations,          # Overlay the significance stars
            fmt='s',                    # Format as a string
            cmap='vlag',                # A good diverging colormap (blue for negative, red for positive)
            center=0,
            linewidths=.5,
            cbar_kws={'label': 'Mean Cumulative Abnormal Return (CAR)'} # Color bar label
        )

        # --- Chart Enhancement ---
        plt.title(f'Heatmap of Mean CAR Significance\nModel: {model_to_plot}, Window: {window}', fontsize=16)
        plt.xlabel('Event Group', fontsize=12)
        plt.ylabel('Asset', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        
        # Display each plot in the correct order
        plt.show()